In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import datetime as dt

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
primaryData = pd.read_csv('./data/train_data/train_task_1_2.csv')
studentMetadata = pd.read_csv('./data/metadata/student_metadata_task_1_2.csv')
answerMetadata = pd.read_csv('./data/metadata/answer_metadata_task_1_2.csv')
questionSubject = pd.read_csv('question_subject_metadata.csv')

In [4]:
# Assigning NaN value to all date of birth that is lower than 2001 or higher than 2013
def cleanDob(cell) :
    if not(pd.isna(cell)) :
        dob = int(cell.split("-")[0])
        if (dob < 2001) or (dob > 2013):
            return float("nan")
    return cell
            
studentMetadata["DateOfBirth"] = studentMetadata["DateOfBirth"].apply(lambda cell: cleanDob(cell))

In [5]:
# Parsing DateAnswered & DateOfBirth from String to DateTime
answerMetadata["DateAnswered"] = pd.to_datetime(answerMetadata["DateAnswered"])
studentMetadata["DateOfBirth"] = pd.to_datetime(studentMetadata["DateOfBirth"])

In [6]:
# Merging PrimaryData with StudentMetaData on column (UserId)
# Merging AnswerMetaData with the output of merged PrimaryData & StudentMetaData
primaryWithStudent = pd.merge(primaryData, studentMetadata, on = 'UserId', how = 'inner')
primaryStudentAnswer = pd.merge(answerMetadata, primaryWithStudent, on = 'AnswerId', how = 'inner')

In [7]:
# Saving DateOfBirth and GroupId pair in a dictionary
# Students with the same GroupId are assumed to have the same date of birth
# Assigning the empty DateOfBirth cells with a GroupId that is stored in the dictionary with the date of birth
# If there is any empty cells remain, assign the mean of date of birth
groupIdDob = {}

def dobKeyValue(row) :
    if row.GroupId not in groupIdDob :
        if not(pd.isnull(row.DateOfBirth)) :
            groupIdDob[id] = row.DateOfBirth
            
def populateDob(dob, groupId, mean) :
    if pd.isna(dob) :
        if groupId in groupIdDob :
            return groupIdDob[groupId]
        else :
            return mean
    return dob

primaryStudentAnswer.apply(lambda row: dobKeyValue(row), axis = 1)
dobMean = studentMetadata["DateOfBirth"].mean()
primaryStudentAnswer["DateOfBirth"] = primaryStudentAnswer.apply(lambda row: populateDob(row["DateOfBirth"], row["GroupId"], dobMean), axis = 1)

In [8]:
# Creating new Age column by subtracting the year the question answered with year of birth
# Getting the Day, Month, Year, Hour and the minute of the answer
# Assigning (1) for questions answered before 2:00 PM as a mornining shift and assign (2) for night shift
primaryStudentAnswer = primaryStudentAnswer.assign(Age = (pd.DatetimeIndex(primaryStudentAnswer["DateAnswered"]).year 
                                                        - pd.DatetimeIndex(primaryStudentAnswer["DateOfBirth"]).year))

primaryStudentAnswer["YearAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell: cell.year)
primaryStudentAnswer["MonthAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell: cell.month)
primaryStudentAnswer["DayAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell: cell.day)
primaryStudentAnswer["HourAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell: cell.hour)
primaryStudentAnswer["MinuteAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell: cell.minute)
primaryStudentAnswer["TimeAnswered"] = primaryStudentAnswer["DateAnswered"].apply(lambda cell : cell.time())
primaryStudentAnswer["SchoolShift"] = primaryStudentAnswer["TimeAnswered"].apply(lambda cell: 1 if cell.hour < 14 else 2)

In [9]:
primaryStudentAnswer = primaryStudentAnswer.sort_values(by=["QuizId", "UserId", "YearAnswered", "MonthAnswered",
                                                            "DayAnswered", "HourAnswered", "MinuteAnswered"])

In [10]:
# Sorting all answers by QuizId, UserId and TimeAnswered ascending
# To get the time to answer the question in minutes
# primaryStudentAnswer = primaryStudentAnswer.sort_values(by=["QuizId", "UserId", "TimeAnswered"])
quizUser = {}
currentTime = primaryStudentAnswer.iloc[0]["TimeAnswered"]
currentDate = primaryStudentAnswer.iloc[0]["DateAnswered"]
totalCost = 0
data = []
qId, uId = 0, 0

def calculateTimeToSolveQuestion(row) :
    global currentDate, currentTime, totalCost, qId, uId
    if row.QuizId in quizUser :
        if quizUser[row.QuizId] == row.UserId :
            if (currentDate.day - row.DateAnswered.day == 0 and 
                currentDate.month - row.DateAnswered.month == 0 and 
                currentDate.year - row.DateAnswered.year == 0) :
                cost = (datetime.combine(date.today(), row.TimeAnswered) - datetime.combine(date.today(), currentTime)).seconds // 60
                currentTime = row.TimeAnswered
                currentDate = row.DateAnswered
                totalCost += cost
                return cost
    if totalCost != 0 :
        data.append([qId, uId, totalCost + 1])
        totalCost = 0
    qId = row.QuizId
    uId = row.UserId
    quizUser[row.QuizId] = row.UserId
    currentTime = row.TimeAnswered
    currentDate = row.DateAnswered
    return 1

primaryStudentAnswer["TimeToAnswerQuestion"] = primaryStudentAnswer.apply(lambda row: calculateTimeToSolveQuestion(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'TimeToAnswerQuiz'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
primaryStudentAnswer = pd.merge(primaryStudentAnswer, df, on=["QuizId", "UserId"], how="left")
# primaryStudentAnswer.TimeToAnswerQuiz.fillna(1.0)

In [11]:
# primaryStudentAnswer = primaryStudentAnswer.sort_values(by=["QuizId", "UserId", "TimeAnswered"])
# primaryStudentAnswer = primaryStudentAnswer.sort_values(by=["QuizId", "UserId", "DateAnswered"])
quizUser = {}
totalCost = 1
data = []
qId, uId = 0, 0
currentDate = primaryStudentAnswer.iloc[0]["DateAnswered"]

def calculateQuizLength(row) :
    global currentDate, totalCost, qId, uId
    if row.QuizId in quizUser :
        if quizUser[row.QuizId] == row.UserId :
            if (currentDate.day - row.DateAnswered.day == 0 and 
                currentDate.month - row.DateAnswered.month == 0 and 
                currentDate.year - row.DateAnswered.year == 0) :
                totalCost += 1
            else :
                data.append([qId, uId, totalCost])
                totalCost = 1
        elif totalCost > 1 :
            data.append([qId, uId, totalCost])
            totalCost = 1
    qId = row.QuizId
    uId = row.UserId
    quizUser[row.QuizId] = row.UserId
    currentDate = row.DateAnswered

primaryStudentAnswer.apply(lambda row: calculateQuizLength(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'QuizLength'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
primaryStudentAnswer = pd.merge(primaryStudentAnswer, df, on=["QuizId", "UserId"], how="left")
# primaryStudentAnswer.QuizLength.fillna(10, inplace=True)

In [12]:
primaryStudentAnswer["QuizLength"] = primaryStudentAnswer["QuizLength"].apply(lambda row: 30 if row > 30 else row)
primaryStudentAnswer["TimeToAnswerQuestion"] = primaryStudentAnswer["TimeToAnswerQuestion"].apply(lambda row: 30 if row > 30 else row)

In [13]:
quizTimeMean = {}
questionTimeMean = {}

def calculateMean(row) :
    if row.QuizId not in quizTimeMean :
        df = primaryStudentAnswer[primaryStudentAnswer["QuizId"] == row.QuizId]
        quizTimeMean[row.QuizId] = df["TimeToAnswerQuiz"].median()
    if row.QuestionId not in questionTimeMean :
        df = primaryStudentAnswer[primaryStudentAnswer["QuestionId"] == row.QuestionId]
        questionTimeMean[row.QuestionId] = df["TimeToAnswerQuestion"].median()

primaryStudentAnswer.apply(lambda row: calculateMean(row), axis = 1)
primaryStudentAnswer["QuestionMedianTime"] = primaryStudentAnswer["QuestionId"].apply(lambda x: questionTimeMean[x])
primaryStudentAnswer["QuizMedianTime"] = primaryStudentAnswer["QuizId"].apply(lambda x: quizTimeMean[x])

In [15]:
schemeAge = {}

def getSchemeOfWorkIds(row) :
    if not(pd.isnull(row.SchemeOfWorkId)) :
        if not(row.Age in schemeAge) :
            schemeAge[row.Age] = row.SchemeOfWorkId

primaryStudentAnswer.apply(lambda row: getSchemeOfWorkIds(row), axis = 1)
primaryStudentAnswer["SchemeOfWorkId"] = primaryStudentAnswer.apply(lambda row: schemeAge[row.Age] if pd.isnull(row.SchemeOfWorkId) else row.SchemeOfWorkId, axis = 1)

In [ ]:
primaryStudentAnswer.isna().sum()

In [20]:
# Filling NaN values in Confidence with 50, 0 in PremiumPupil and the mean of SchemeOfWorkId in SchemeOfWorkId
primaryStudentAnswer["QuizMedianTime"] = primaryStudentAnswer["QuizMedianTime"].fillna(primaryStudentAnswer["QuizMedianTime"].median())
primaryStudentAnswer["QuizLength"] = primaryStudentAnswer["QuizLength"].fillna(10)
primaryStudentAnswer["TimeToAnswerQuiz"] = primaryStudentAnswer["TimeToAnswerQuiz"].fillna(primaryStudentAnswer["TimeToAnswerQuiz"].mean())
primaryStudentAnswer["Confidence"] = primaryStudentAnswer["Confidence"].fillna(50)
primaryStudentAnswer["PremiumPupil"] = primaryStudentAnswer["PremiumPupil"].fillna(0)

In [5]:
# Merging all the data
allData = pd.merge(primaryStudentAnswer, questionSubject, on = "QuestionId", how = "left")

In [6]:
# Calculating the number of questions given for each student
numOfQstForStd = {}

def condition(userId) :
    if(userId in numOfQstForStd) :
        numOfQstForStd[userId] += 1
    else :
        numOfQstForStd[userId] = 1

allData["UserId"].apply(lambda row: condition(row))
allData["#QuestionForStudent"] = allData["UserId"].apply(lambda row: numOfQstForStd[row])

In [7]:
# Calculating the number of correct answers given for each question
# Calculating the number of correct answers given for each student
numOfCorrectAnsForStd = {}
numOfCorrectAnsForQst = {}

def condition(row) :
    if(row["IsCorrect"] == 1) :
        if(row["UserId"] in numOfCorrectAnsForStd) :
            numOfCorrectAnsForStd[row["UserId"]] += 1
        else :
            numOfCorrectAnsForStd[row["UserId"]] = 1
        if(row["QuestionId"] in numOfCorrectAnsForQst) :
            numOfCorrectAnsForQst[row["QuestionId"]] += 1
        else :
            numOfCorrectAnsForQst[row["QuestionId"]] = 1

allData.apply(lambda row: condition(row), axis = 1)

0           None
1           None
2           None
3           None
4           None
            ... 
15867845    None
15867846    None
15867847    None
15867848    None
15867849    None
Length: 15867850, dtype: object

In [ ]:
# Calculating the IQ Level for each student by dividing the #correct answers over all answers
allData["IQLevel"] = allData.apply(lambda row: numOfCorrectAnsForStd[row["UserId"]] / row["#QuestionForStudent"], axis = 1)

In [8]:
# Calculating the IQ Level for each student by dividing the #correct answers over all answers
allData["IQLevel"] = 0.0

def condition(row) :
    return numOfCorrectAnsForStd[row["UserId"]] / row["#QuestionForStudent"]

allData["IQLevel"] = allData.apply(lambda row: condition(row), axis = 1)

In [9]:
# Calculating the number of answers given for each question
# Calculating the question difficulty by dividing the correct answers over all answers
allData["QuestionDifficulty"] = 0.0
numOfAnsForQst = {}

def calculateNumOfAns(questionId) :
    if(questionId in numOfAnsForQst) :
        numOfAnsForQst[questionId] += 1
    else :
        numOfAnsForQst[questionId] = 1

def calculateQstDiff(question) :
    return numOfCorrectAnsForQst[question] / numOfAnsForQst[question]

allData["QuestionId"].apply(lambda row: calculateNumOfAns(row))
allData["QuestionDifficulty"] = allData["QuestionId"].apply(lambda row: calculateQstDiff(row))

In [10]:
# Calculating the most answer for each question
# pd.Series.mode operation returns np.ndarray if the to answers were picked equaly
# Removing the array by returning the first element of it
def parseCommonAnswer(row) :
    if isinstance(row, np.ndarray) :
        return row[0]
    return row

df = allData.groupby(["QuestionId"])["AnswerValue"].agg(pd.Series.mode)
allData["CommonAnswer"] = allData["QuestionId"].apply(lambda row: df[row])
allData["CommonAnswer"] = allData["CommonAnswer"].apply(lambda row: parseCommonAnswer(row))

In [11]:
for i in range(1,9) :
    math = "{}{}".format("Math", i)
    numOfAnswers = "{}{}{}".format("#", math, "Answers")
    numOfCorrectAnswers = "{}{}{}".format("#", math, "CorrectAnswers")
    numOfStdAnswers = "{}{}{}".format("#", math, "StdAnswers")
    numOfStdCorrectAnswers = "{}{}{}".format("#", math, "StdCorrectAnswers")
    (allData[numOfAnswers], allData[numOfCorrectAnswers],
    allData[numOfStdAnswers], allData[numOfStdCorrectAnswers]) = (0, 0, 0, 0)

In [12]:
for i in range(1, 9) :
    math = "{}{}".format("Math", i)
    mathDiff = "{}{}".format(math, "Difficulty")
    numOfAnswer = "{}{}{}".format("#", math, "Answers")
    numOfCorrectAnswer = "{}{}{}".format("#", math, "CorrectAnswers")
    df = allData[allData[math] == 1]
    numOfAnswers = df.shape[0]
    numOfCorrectAnswers = df["IsCorrect"].isin([1]).sum()
    diff = numOfCorrectAnswers / numOfAnswers
    allData[mathDiff] = allData[math].apply(lambda cell: diff)
    allData[numOfAnswer] = allData[math].apply(lambda cell: numOfAnswers)
    allData[numOfCorrectAnswer] = allData[math].apply(lambda cell: numOfCorrectAnswers)

In [13]:
numStdCorrect = {}
numStdAnswer = {}

def condition(row) :
    if row.UserId in numStdAnswer :
        numStdAnswer[row.UserId] += 1
    else :
        numStdAnswer[row.UserId] = 1
        
    if row.IsCorrect == 1 :
        if row.UserId in numStdCorrect :
            numStdCorrect[row.UserId] += 1
        else :
            numStdCorrect[row.UserId] = 1

def calculateIq(row, math) :
    if row.UserId in numStdCorrect and row.UserId in numStdAnswer :
        return numStdCorrect[row.UserId] / numStdAnswer[row.UserId]
    return 0

for i in range(1, 9) :
    numStdCorrect = {}
    numStdAnswer = {}
    math = "{}{}".format("Math", i)
    mathIQ = "{}{}".format(math, "StdIQ")
    numOfStdAnswers = "{}{}{}".format("#", math, "StdAnswers")
    numOfStdCorrectAnswers = "{}{}{}".format("#", math, "StdCorrectAnswers")
    df = allData[allData[math] == 1]
    
    df.apply(lambda row: condition(row), axis = 1)
    allData[mathIQ] = allData.apply(lambda row: calculateIq(row, math), axis = 1)
    allData[numOfStdAnswers] = allData.apply(lambda row: numStdAnswer[row["UserId"]], axis = 1)
    allData[numOfStdCorrectAnswers] = allData.apply(lambda row: numStdCorrect[row["UserId"]], axis = 1)

KeyError: 3342

In [14]:
for i in range(1, 9) :
    math = "{}{}".format("Math", i)
    answer = "{}{}{}".format("#",math, "Answers")
    correct = "{}{}{}".format("#", math, "CorrectAnswers")
    stdanswer = "{}{}{}".format("#",math, "StdAnswers")
    stdcorrect = "{}{}{}".format("#", math, "StdCorrectAnswers")
    lvl = "{}{}".format(math, "Level")
    allData.drop([answer, correct, stdanswer, stdcorrect, lvl],  inplace = True, axis = 1)

# allData.drop(["QuestionLevel", "#QuestionForStudent", "DateOfBirth",
#               "DateAnswered", "TimeAnswered"], inplace = True, axis = 1)

allData.drop(["DateOfBirth", "DateAnswered"], inplace = True, axis = 1)

In [15]:
allData = allData.sample(frac=1).reset_index(drop=True)
allData

,AnswerId,Confidence,GroupId,QuizId,SchemeOfWorkId,QuestionId,UserId,IsCorrect,CorrectAnswer,AnswerValue,Gender,PremiumPupil,Age,YearAnswered,MonthAnswered,DayAnswered,HourAnswered,MinuteAnswered,TimeAnswered,SchoolShift,TimeToAnswerQuestion,TimeToAnswerQuiz,QuizLength,QuestionMedianTime,QuizMedianTime,Math1,Math1StdIQ,Math1Difficulty,Math2,Math2StdIQ,Math2Difficulty,Math3,Math3StdIQ,Math3Difficulty,Math4,Math4StdIQ,Math4Difficulty,Math5,Math5StdIQ,Math5Difficulty,Math6,Math6StdIQ,Math6Difficulty,Math7,Math7StdIQ,Math7Difficulty,Math8,Math8StdIQ,Math8Difficulty,QuestionLevel,QuestionDifficulty,#QuestionForStudent,IQLevel,CommonAnswer
0,16315106.0,50.0,3332,6385,8408.0,285,37321,0,1,4,2,0.0,14,2020,2,11,9,50,09:50:00,1,0,17.85845,9.0,1.0,6.0,0,0.166667,0.66824,0,0,0.611376,1,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.671681,103,0.368932,1
1,8651493.0,50.0,3759,5301,8413.0,15698,47052,0,4,3,1,0.0,16,2019,10,4,8,33,08:33:00,1,1,16.00000,9.0,1.0,9.0,0,0.500000,0.66824,0,0,0.611376,0,0,0.622576,1,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,9,0.512723,245,0.420408,4
2,8613789.0,50.0,2580,4054,66488.0,26366,24625,1,2,2,0,0.0,13,2019,11,10,17,36,17:36:00,2,0,4.00000,17.0,0.0,9.0,1,0.720930,0.66824,1,0,0.611376,0,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,15,0.489091,147,0.727891,2
3,19493915.0,50.0,1482,14864,8386.0,16354,52136,0,2,3,0,0.0,12,2018,11,8,9,40,09:40:00,1,6,16.00000,6.0,1.0,9.0,1,0.376000,0.66824,0,0,0.611376,0,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.705657,129,0.372093,2
4,11182099.0,50.0,360,1314,8362.0,6417,75466,0,3,2,2,0.0,15,2020,1,10,13,59,13:59:00,1,1,7.00000,10.0,1.0,8.0,0,0.846154,0.66824,1,0,0.611376,0,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.625210,95,0.863158,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15867845,15057635.0,100.0,6231,6059,28237.0,3897,38461,1,2,2,0,0.0,13,2019,6,6,18,34,18:34:00,2,1,65.00000,12.0,1.0,8.0,1,0.901408,0.66824,0,0,0.611376,0,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.440786,394,0.802030,2
15867846,13101127.0,50.0,7174,3374,8412.0,12287,33077,1,2,2,2,0.0,15,2019,11,14,17,34,17:34:00,2,0,4.00000,9.0,0.0,5.0,0,0.788136,0.66824,0,0,0.611376,1,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.639216,350,0.728571,2
15867847,1929029.0,50.0,2035,3812,28228.0,25944,57179,0,2,4,1,0.0,13,2018,10,18,18,32,18:32:00,2,0,13.00000,12.0,1.0,8.0,1,0.425234,0.66824,0,0,0.611376,0,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.544304,435,0.409195,2
15867848,1704090.0,50.0,9363,2038,28237.0,20736,13308,0,1,3,2,0.0,16,2020,1,10,12,43,12:43:00,1,0,17.85845,8.0,1.0,14.0,0,0.349515,0.66824,0,0,0.611376,1,0,0.622576,0,0,0.629661,0,0,0.615813,0,0,0.599398,0,0,0.521008,0,0,0.685953,6,0.781884,597,0.308208,1


In [ ]:
allData.to_csv('allData.csv', header=True, index=False)

__________

Preprocess test files

________

In [3]:
allData = pd.read_csv('allData.csv')
answerMetadata = pd.read_csv('./data/metadata/answer_metadata_task_1_2.csv')
studentMetadata = pd.read_csv('./data/metadata/student_metadata_task_1_2.csv')
public_test_task1_data = pd.read_csv('./data/test_data/test_public_answers_task_1.csv')
private_test_task1_data = pd.read_csv('./data/test_data/test_private_answers_task_1.csv')
public_test_task2_data = pd.read_csv('./data/test_data/test_public_answers_task_2.csv')
private_test_task2_data = pd.read_csv('./data/test_data/test_private_answers_task_2.csv')

In [4]:
files = np.array([public_test_task1_data, private_test_task1_data, public_test_task2_data, private_test_task2_data])

/var/folders/_6/6nscwj3n52n621rsv2q3v8_00000gn/T/ipykernel_1012/2814463456.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  files = np.array([public_test_task1_data, private_test_task1_data, public_test_task2_data, private_test_task2_data])


In [14]:
df = allData[[
    "UserId", "IQLevel", "Age",
    "Math1StdIQ", "Math2StdIQ", 
    "Math3StdIQ", "Math4StdIQ",
    "Math5StdIQ", "Math6StdIQ", 
    "Math7StdIQ", "Math8StdIQ",
    "#QuestionForStudent"
]]

df = df.drop_duplicates(subset ="UserId")
public_test_task1_data = pd.merge(public_test_task1_data, df, on = "UserId", how = "inner")
private_test_task1_data = pd.merge(private_test_task1_data, df, on = "UserId", how = "inner")
public_test_task2_data = pd.merge(public_test_task2_data, df, on = "UserId", how = "inner")
private_test_task2_data = pd.merge(private_test_task2_data, df, on = "UserId", how = "inner")

In [15]:
df = allData[[
    "QuestionId", 
    "Math1", "Math1Difficulty",
    "Math2", "Math2Difficulty",
    "Math3", "Math3Difficulty",
    "Math4", "Math4Difficulty",
    "Math5", "Math5Difficulty",
    "Math6", "Math6Difficulty",
    "Math7", "Math7Difficulty",
    "Math8", "Math8Difficulty",
    "QuestionDifficulty",
    "CorrectAnswer", "CommonAnswer",
    "QuestionLevel"
]]

df = df.drop_duplicates(subset ="QuestionId")
public_test_task1_data = pd.merge(public_test_task1_data, df, on = "QuestionId", how = "inner")
private_test_task1_data = pd.merge(private_test_task1_data, df, on = "QuestionId", how = "inner")
public_test_task2_data = pd.merge(public_test_task2_data, df, on = "QuestionId", how = "inner")
private_test_task2_data = pd.merge(private_test_task2_data, df, on = "QuestionId", how = "inner")

del public_test_task1_data["CorrectAnswer"]
del private_test_task1_data["CorrectAnswer"]

In [16]:
public_test_task1_data = pd.merge(public_test_task1_data, studentMetadata, on = 'UserId', how = 'inner')
public_test_task1_data = pd.merge(public_test_task1_data, answerMetadata, on = 'AnswerId', how = 'inner')

private_test_task1_data = pd.merge(private_test_task1_data, studentMetadata, on = 'UserId', how = 'inner')
private_test_task1_data = pd.merge(private_test_task1_data, answerMetadata, on = 'AnswerId', how = 'inner')

public_test_task2_data = pd.merge(public_test_task2_data, studentMetadata, on = 'UserId', how = 'inner')
public_test_task2_data = pd.merge(public_test_task2_data, answerMetadata, on = 'AnswerId', how = 'inner')

private_test_task2_data = pd.merge(private_test_task2_data, studentMetadata, on = 'UserId', how = 'inner')
private_test_task2_data = pd.merge(private_test_task2_data, answerMetadata, on = 'AnswerId', how = 'inner')

In [17]:
def cleanDob(cell) :
    if not(pd.isna(cell)) :
        dob = int(cell.split("-")[0])
        if (dob < 2001) or (dob > 2013):
            return float("nan")
    return cell

public_test_task1_data["DateOfBirth"] = public_test_task1_data["DateOfBirth"].apply(lambda cell: cleanDob(cell))
private_test_task1_data["DateOfBirth"] = private_test_task1_data["DateOfBirth"].apply(lambda cell: cleanDob(cell))
public_test_task2_data["DateOfBirth"] = public_test_task2_data["DateOfBirth"].apply(lambda cell: cleanDob(cell))
private_test_task2_data["DateOfBirth"] = private_test_task2_data["DateOfBirth"].apply(lambda cell: cleanDob(cell))

public_test_task1_data["DateAnswered"] = pd.to_datetime(public_test_task1_data["DateAnswered"])
public_test_task1_data["DateOfBirth"] = pd.to_datetime(public_test_task1_data["DateOfBirth"])

private_test_task1_data["DateAnswered"] = pd.to_datetime(private_test_task1_data["DateAnswered"])
private_test_task1_data["DateOfBirth"] = pd.to_datetime(private_test_task1_data["DateOfBirth"])

public_test_task2_data["DateAnswered"] = pd.to_datetime(public_test_task2_data["DateAnswered"])
public_test_task2_data["DateOfBirth"] = pd.to_datetime(public_test_task2_data["DateOfBirth"])

private_test_task2_data["DateAnswered"] = pd.to_datetime(private_test_task2_data["DateAnswered"])
private_test_task2_data["DateOfBirth"] = pd.to_datetime(private_test_task2_data["DateOfBirth"])

In [18]:
groupIdDob = {}

def dobKeyValue(row) :
    if row.GroupId not in groupIdDob :
        if not(pd.isnull(row.DateOfBirth)) :
            groupIdDob[id] = row.DateOfBirth
            
def populateDob(dob, groupId, mean) :
    if pd.isna(dob) :
        if groupId in groupIdDob :
            return groupIdDob[groupId]
        else :
            return mean
    return dob

public_test_task1_data.apply(lambda row: dobKeyValue(row), axis = 1)
dobMean = public_test_task1_data["DateOfBirth"].mean()

public_test_task1_data["DateOfBirth"] = public_test_task1_data.apply(lambda row: populateDob(row["DateOfBirth"], row["GroupId"], dobMean), axis = 1)
private_test_task1_data["DateOfBirth"] = private_test_task1_data.apply(lambda row: populateDob(row["DateOfBirth"], row["GroupId"], dobMean), axis = 1)
public_test_task2_data["DateOfBirth"] = public_test_task2_data.apply(lambda row: populateDob(row["DateOfBirth"], row["GroupId"], dobMean), axis = 1)
private_test_task2_data["DateOfBirth"] = private_test_task2_data.apply(lambda row: populateDob(row["DateOfBirth"], row["GroupId"], dobMean), axis = 1)

In [19]:
public_test_task1_data["TimeAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell : cell.time())
public_test_task1_data["SchoolShift"] = public_test_task1_data["TimeAnswered"].apply(lambda cell: 1 if cell.hour < 14 else 2)

private_test_task1_data["TimeAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell : cell.time())
private_test_task1_data["SchoolShift"] = private_test_task1_data["TimeAnswered"].apply(lambda cell: 1 if cell.hour < 14 else 2)

public_test_task2_data["TimeAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell : cell.time())
public_test_task2_data["SchoolShift"] = public_test_task2_data["TimeAnswered"].apply(lambda cell: 1 if cell.hour < 14 else 2)

private_test_task2_data["TimeAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell : cell.time())
private_test_task2_data["SchoolShift"] = private_test_task2_data["TimeAnswered"].apply(lambda cell: 1 if cell.hour < 14 else 2)

In [20]:
public_test_task1_data["YearAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell: cell.year)
public_test_task1_data["MonthAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell: cell.month)
public_test_task1_data["DayAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell: cell.day)
public_test_task1_data["HourAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell: cell.hour)
public_test_task1_data["MinuteAnswered"] = public_test_task1_data["DateAnswered"].apply(lambda cell: cell.minute)

private_test_task1_data["YearAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell: cell.year)
private_test_task1_data["MonthAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell: cell.month)
private_test_task1_data["DayAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell: cell.day)
private_test_task1_data["HourAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell: cell.hour)
private_test_task1_data["MinuteAnswered"] = private_test_task1_data["DateAnswered"].apply(lambda cell: cell.minute)

public_test_task2_data["YearAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell: cell.year)
public_test_task2_data["MonthAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell: cell.month)
public_test_task2_data["DayAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell: cell.day)
public_test_task2_data["HourAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell: cell.hour)
public_test_task2_data["MinuteAnswered"] = public_test_task2_data["DateAnswered"].apply(lambda cell: cell.minute)

private_test_task2_data["YearAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell: cell.year)
private_test_task2_data["MonthAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell: cell.month)
private_test_task2_data["DayAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell: cell.day)
private_test_task2_data["HourAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell: cell.hour)
private_test_task2_data["MinuteAnswered"] = private_test_task2_data["DateAnswered"].apply(lambda cell: cell.minute)

In [21]:
# Sorting all answers by QuizId, UserId and TimeAnswered ascending
# To get the time to answer the question in minutes

public_test_task1_data = public_test_task1_data.sort_values(by=["QuizId", "UserId", "YearAnswered", "MonthAnswered",
                                                            "DayAnswered", "HourAnswered", "MinuteAnswered"])
private_test_task1_data = private_test_task1_data.sort_values(by=["QuizId", "UserId", "YearAnswered", "MonthAnswered",
                                                            "DayAnswered", "HourAnswered", "MinuteAnswered"])
public_test_task2_data = public_test_task2_data.sort_values(by=["QuizId", "UserId", "YearAnswered", "MonthAnswered",
                                                            "DayAnswered", "HourAnswered", "MinuteAnswered"])
private_test_task2_data = private_test_task2_data.sort_values(by=["QuizId", "UserId", "YearAnswered", "MonthAnswered",
                                                            "DayAnswered", "HourAnswered", "MinuteAnswered"])

quizUser = {}
currentTime = public_test_task1_data.iloc[0]["TimeAnswered"]
currentDate = public_test_task1_data.iloc[0]["DateAnswered"]
totalCost = 0
data = []
qId, uId = 0, 0

def calculateTimeToSolveQuestion(row) :
    global currentDate, currentTime, totalCost, qId, uId
    if row.QuizId in quizUser :
        if quizUser[row.QuizId] == row.UserId :
            if (currentDate.day - row.DateAnswered.day == 0 and 
                currentDate.month - row.DateAnswered.month == 0 and 
                currentDate.year - row.DateAnswered.year == 0) :
                cost = (datetime.combine(date.today(), row.TimeAnswered) - datetime.combine(date.today(), currentTime)).seconds // 60
                currentTime = row.TimeAnswered
                currentDate = row.DateAnswered
                totalCost += cost
                return cost
    if totalCost != 0 :
        data.append([qId, uId, totalCost + 1])
        totalCost = 0
    qId = row.QuizId
    uId = row.UserId
    quizUser[row.QuizId] = row.UserId
    currentTime = row.TimeAnswered
    currentDate = row.DateAnswered
    return 1

public_test_task1_data["TimeToAnswerQuestion"] = public_test_task1_data.apply(lambda row: calculateTimeToSolveQuestion(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'TimeToAnswerQuiz'])
public_test_task1_data["TimeToAnswerQuiz"] = df["TimeToAnswerQuiz"]

quizUser = {}
currentTime = private_test_task1_data.iloc[0]["TimeAnswered"]
currentDate = private_test_task1_data.iloc[0]["DateAnswered"]
data = []

private_test_task1_data["TimeToAnswerQuestion"] = private_test_task1_data.apply(lambda row: calculateTimeToSolveQuestion(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'TimeToAnswerQuiz'])
private_test_task1_data["TimeToAnswerQuiz"] = df["TimeToAnswerQuiz"]

quizUser = {}
currentTime = public_test_task2_data.iloc[0]["TimeAnswered"]
currentDate = public_test_task2_data.iloc[0]["DateAnswered"]
data = []

public_test_task2_data["TimeToAnswerQuestion"] = public_test_task2_data.apply(lambda row: calculateTimeToSolveQuestion(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'TimeToAnswerQuiz'])
public_test_task2_data["TimeToAnswerQuiz"] = df["TimeToAnswerQuiz"]

quizUser = {}
currentTime = private_test_task2_data.iloc[0]["TimeAnswered"]
currentDate = private_test_task2_data.iloc[0]["DateAnswered"]
data = []

private_test_task2_data["TimeToAnswerQuestion"] = private_test_task2_data.apply(lambda row: calculateTimeToSolveQuestion(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'TimeToAnswerQuiz'])
private_test_task2_data["TimeToAnswerQuiz"] = df["TimeToAnswerQuiz"]

In [22]:
quizUser = {}
totalCost = 1
data = []
qId, uId = 0, 0
currentDate = public_test_task1_data.iloc[0]["DateAnswered"]

def calculateQuizLength(row) :
    global currentDate, totalCost, qId, uId
    if row.QuizId in quizUser :
        if quizUser[row.QuizId] == row.UserId :
            if (currentDate.day - row.DateAnswered.day == 0 and 
                currentDate.month - row.DateAnswered.month == 0 and 
                currentDate.year - row.DateAnswered.year == 0) :
                totalCost += 1
            else :
                data.append([qId, uId, totalCost])
                totalCost = 1
        elif totalCost > 1 :
            data.append([qId, uId, totalCost])
            totalCost = 1
    qId = row.QuizId
    uId = row.UserId
    quizUser[row.QuizId] = row.UserId
    currentDate = row.DateAnswered

public_test_task1_data.apply(lambda row: calculateQuizLength(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'QuizLength'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
public_test_task1_data = pd.merge(public_test_task1_data, df, on=["QuizId", "UserId"], how="left")

quizUser = {}
totalCost = 1
data = []
qId, uId = 0, 0
currentDate = private_test_task1_data.iloc[0]["DateAnswered"]

private_test_task1_data.apply(lambda row: calculateQuizLength(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'QuizLength'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
private_test_task1_data = pd.merge(private_test_task1_data, df, on=["QuizId", "UserId"], how="left")

quizUser = {}
totalCost = 1
data = []
qId, uId = 0, 0
currentDate = public_test_task2_data.iloc[0]["DateAnswered"]

public_test_task2_data.apply(lambda row: calculateQuizLength(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'QuizLength'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
public_test_task2_data = pd.merge(public_test_task2_data, df, on=["QuizId", "UserId"], how="left")

quizUser = {}
totalCost = 1
data = []
qId, uId = 0, 0
currentDate = private_test_task2_data.iloc[0]["DateAnswered"]

private_test_task2_data.apply(lambda row: calculateQuizLength(row), axis = 1)
df = pd.DataFrame(data, columns =['QuizId', 'UserId', 'QuizLength'])
df.drop_duplicates(subset=['QuizId', 'UserId'], inplace=True)
private_test_task2_data = pd.merge(private_test_task2_data, df, on=["QuizId", "UserId"], how="left")

In [23]:
public_test_task1_data["QuizLength"] = public_test_task1_data["QuizLength"].apply(lambda row: 30 if row > 30 else row)
public_test_task1_data["TimeToAnswerQuestion"] = public_test_task1_data["TimeToAnswerQuestion"].apply(lambda row: 30 if row > 30 else row)

private_test_task1_data["QuizLength"] = private_test_task1_data["QuizLength"].apply(lambda row: 30 if row > 30 else row)
private_test_task1_data["TimeToAnswerQuestion"] = private_test_task1_data["TimeToAnswerQuestion"].apply(lambda row: 30 if row > 30 else row)

public_test_task2_data["QuizLength"] = public_test_task2_data["QuizLength"].apply(lambda row: 30 if row > 30 else row)
public_test_task2_data["TimeToAnswerQuestion"] = public_test_task2_data["TimeToAnswerQuestion"].apply(lambda row: 30 if row > 30 else row)

private_test_task2_data["QuizLength"] = private_test_task2_data["QuizLength"].apply(lambda row: 30 if row > 30 else row)
private_test_task2_data["TimeToAnswerQuestion"] = private_test_task2_data["TimeToAnswerQuestion"].apply(lambda row: 30 if row > 30 else row)

In [25]:
quizTimeMean = {}
questionTimeMean = {}

def calculateMean(row, file) :
    global quizTimeMean, questionTimeMean
    if row.QuizId not in quizTimeMean :
        df = file[file["QuizId"] == row.QuizId]
        quizTimeMean[row.QuizId] = df["TimeToAnswerQuiz"].median()
    if row.QuestionId not in questionTimeMean :
        df = file[file["QuestionId"] == row.QuestionId]
        questionTimeMean[row.QuestionId] = df["TimeToAnswerQuestion"].median()

quizTimeMean = {}
questionTimeMean = {}
public_test_task1_data.apply(lambda row: calculateMean(row, public_test_task1_data), axis = 1)
public_test_task1_data["QuestionMedianTime"] = public_test_task1_data["QuestionId"].apply(lambda x: questionTimeMean[x])
public_test_task1_data["QuizMedianTime"] = public_test_task1_data["QuizId"].apply(lambda x: quizTimeMean[x])

quizTimeMean = {}
questionTimeMean = {}
private_test_task1_data.apply(lambda row: calculateMean(row, private_test_task1_data), axis = 1)
private_test_task1_data["QuestionMedianTime"] = private_test_task1_data["QuestionId"].apply(lambda x: questionTimeMean[x])
private_test_task1_data["QuizMedianTime"] = private_test_task1_data["QuizId"].apply(lambda x: quizTimeMean[x])

quizTimeMean = {}
questionTimeMean = {}
public_test_task2_data.apply(lambda row: calculateMean(row, public_test_task2_data), axis = 1)
public_test_task2_data["QuestionMedianTime"] = public_test_task2_data["QuestionId"].apply(lambda x: questionTimeMean[x])
public_test_task2_data["QuizMedianTime"] = public_test_task2_data["QuizId"].apply(lambda x: quizTimeMean[x])

quizTimeMean = {}
questionTimeMean = {}
private_test_task2_data.apply(lambda row: calculateMean(row, private_test_task2_data), axis = 1)
private_test_task2_data["QuestionMedianTime"] = private_test_task2_data["QuestionId"].apply(lambda x: questionTimeMean[x])
private_test_task2_data["QuizMedianTime"] = private_test_task2_data["QuizId"].apply(lambda x: quizTimeMean[x])

In [27]:
schemeAge = {}

def getSchemeOfWorkIds(row) :
    if not(pd.isnull(row.SchemeOfWorkId)) :
        if not(row.Age in schemeAge) :
            schemeAge[row.Age] = row.SchemeOfWorkId

public_test_task1_data.apply(lambda row: getSchemeOfWorkIds(row), axis = 1)
public_test_task1_data["SchemeOfWorkId"] = public_test_task1_data.apply(lambda row: schemeAge[row.Age] if pd.isnull(row.SchemeOfWorkId) else row.SchemeOfWorkId, axis = 1)
private_test_task1_data["SchemeOfWorkId"] = private_test_task1_data.apply(lambda row: schemeAge[row.Age] if pd.isnull(row.SchemeOfWorkId) else row.SchemeOfWorkId, axis = 1)
public_test_task2_data["SchemeOfWorkId"] = public_test_task2_data.apply(lambda row: schemeAge[row.Age] if pd.isnull(row.SchemeOfWorkId) else row.SchemeOfWorkId, axis = 1)
private_test_task2_data["SchemeOfWorkId"] = private_test_task2_data.apply(lambda row: schemeAge[row.Age] if pd.isnull(row.SchemeOfWorkId) else row.SchemeOfWorkId, axis = 1)

public_test_task1_data["TimeToAnswerQuiz"].fillna(1.0, inplace=True)
private_test_task1_data["TimeToAnswerQuiz"].fillna(1.0, inplace=True)
public_test_task2_data["TimeToAnswerQuiz"].fillna(1.0, inplace=True)
private_test_task2_data["TimeToAnswerQuiz"].fillna(1.0, inplace=True)

public_test_task1_data["Confidence"].fillna(50, inplace=True)
private_test_task1_data["Confidence"].fillna(50, inplace=True)
public_test_task2_data["Confidence"].fillna(50, inplace=True)
private_test_task2_data["Confidence"].fillna(50, inplace=True)

public_test_task1_data["PremiumPupil"].fillna(0, inplace=True)
private_test_task1_data["PremiumPupil"].fillna(0, inplace=True)
public_test_task2_data["PremiumPupil"].fillna(0, inplace=True)
private_test_task2_data["PremiumPupil"].fillna(0, inplace=True)

public_test_task1_data["QuizLength"].fillna(10, inplace=True)
private_test_task1_data["QuizLength"].fillna(10, inplace=True)
public_test_task2_data["QuizLength"].fillna(10, inplace=True)
private_test_task2_data["QuizLength"].fillna(10, inplace=True)

public_test_task1_data["QuizMedianTime"].fillna(10, inplace=True)
private_test_task1_data["QuizMedianTime"].fillna(10, inplace=True)
public_test_task2_data["QuizMedianTime"].fillna(10, inplace=True)
private_test_task2_data["QuizMedianTime"].fillna(10, inplace=True)

In [ ]:
public_test_task1_data.drop(["DateOfBirth", "DateAnswered", "TimeAnswered"],  inplace = True, axis = 1)
private_test_task1_data.drop(["DateOfBirth", "DateAnswered", "TimeAnswered"],  inplace = True, axis = 1)
public_test_task2_data.drop(["DateOfBirth", "DateAnswered", "TimeAnswered"],  inplace = True, axis = 1)
private_test_task2_data.drop(["DateOfBirth", "DateAnswered", "TimeAnswered"],  inplace = True, axis = 1)

In [36]:
allData = allData[[
    "QuestionId", "AnswerId", "UserId", "QuizId","GroupId", "SchemeOfWorkId",
    "IsCorrect", "CorrectAnswer", "AnswerValue", "CommonAnswer",
    "Gender", "Age", "IQLevel", "PremiumPupil", "Confidence",
    "SchoolShift", "TimeToAnswerQuestion", "TimeToAnswerQuiz", 
    "QuestionDifficulty", "QuestionMedianTime", "QuizMedianTime",
    "Math1", "Math1StdIQ", "Math1Difficulty",
    "Math2", "Math2StdIQ", "Math2Difficulty",
    "Math3", "Math3StdIQ", "Math3Difficulty",
    "Math4", "Math4StdIQ", "Math4Difficulty",
    "Math5", "Math5StdIQ", "Math5Difficulty",
    "Math6", "Math6StdIQ", "Math6Difficulty",
    "Math7", "Math7StdIQ", "Math7Difficulty",
    "Math8", "Math8StdIQ", "Math8Difficulty",
    "YearAnswered", "MonthAnswered", "DayAnswered",
    "HourAnswered", "MinuteAnswered", "QuestionLevel",
    "QuizLength", "#QuestionForStudent"
]]

In [37]:
public_test_task1_data = public_test_task1_data[[
    "QuestionId", "AnswerId", "UserId", "QuizId","GroupId", "SchemeOfWorkId",
    "IsCorrect", "CommonAnswer","Gender", "Age", "IQLevel", "PremiumPupil", "Confidence",
    "SchoolShift", "TimeToAnswerQuestion", "TimeToAnswerQuiz", 
    "QuestionDifficulty", "QuestionMedianTime", "QuizMedianTime",
    "Math1", "Math1StdIQ", "Math1Difficulty",
    "Math2", "Math2StdIQ", "Math2Difficulty",
    "Math3", "Math3StdIQ", "Math3Difficulty",
    "Math4", "Math4StdIQ", "Math4Difficulty",
    "Math5", "Math5StdIQ", "Math5Difficulty",
    "Math6", "Math6StdIQ", "Math6Difficulty",
    "Math7", "Math7StdIQ", "Math7Difficulty",
    "Math8", "Math8StdIQ", "Math8Difficulty",
    "YearAnswered", "MonthAnswered", "DayAnswered",
    "HourAnswered", "MinuteAnswered", "QuestionLevel",
    "QuizLength", "#QuestionForStudent"
]]

private_test_task1_data = private_test_task1_data[[
    "QuestionId", "AnswerId", "UserId", "QuizId","GroupId", "SchemeOfWorkId",
    "IsCorrect", "CommonAnswer","Gender", "Age", "IQLevel", "PremiumPupil", "Confidence",
    "SchoolShift", "TimeToAnswerQuestion", "TimeToAnswerQuiz", 
    "QuestionDifficulty", "QuestionMedianTime", "QuizMedianTime",
    "Math1", "Math1StdIQ", "Math1Difficulty",
    "Math2", "Math2StdIQ", "Math2Difficulty",
    "Math3", "Math3StdIQ", "Math3Difficulty",
    "Math4", "Math4StdIQ", "Math4Difficulty",
    "Math5", "Math5StdIQ", "Math5Difficulty",
    "Math6", "Math6StdIQ", "Math6Difficulty",
    "Math7", "Math7StdIQ", "Math7Difficulty",
    "Math8", "Math8StdIQ", "Math8Difficulty",
    "YearAnswered", "MonthAnswered", "DayAnswered",
    "HourAnswered", "MinuteAnswered", "QuestionLevel",
    "QuizLength", "#QuestionForStudent"
]]

In [38]:
public_test_task2_data = public_test_task2_data[[
    "QuestionId", "AnswerId", "UserId", "QuizId","GroupId", "SchemeOfWorkId",
    "CorrectAnswer", "AnswerValue", "CommonAnswer",
    "Gender", "Age", "IQLevel", "PremiumPupil", "Confidence",
    "SchoolShift", "TimeToAnswerQuestion", "TimeToAnswerQuiz", 
    "QuestionDifficulty", "QuestionMedianTime", "QuizMedianTime",
    "Math1", "Math1StdIQ", "Math1Difficulty",
    "Math2", "Math2StdIQ", "Math2Difficulty",
    "Math3", "Math3StdIQ", "Math3Difficulty",
    "Math4", "Math4StdIQ", "Math4Difficulty",
    "Math5", "Math5StdIQ", "Math5Difficulty",
    "Math6", "Math6StdIQ", "Math6Difficulty",
    "Math7", "Math7StdIQ", "Math7Difficulty",
    "Math8", "Math8StdIQ", "Math8Difficulty",
    "YearAnswered", "MonthAnswered", "DayAnswered",
    "HourAnswered", "MinuteAnswered", "QuestionLevel",
    "QuizLength", "#QuestionForStudent"
]]

private_test_task2_data = private_test_task2_data[[
    "QuestionId", "AnswerId", "UserId", "QuizId","GroupId", "SchemeOfWorkId",
    "CorrectAnswer", "AnswerValue", "CommonAnswer",
    "Gender", "Age", "IQLevel", "PremiumPupil", "Confidence",
    "SchoolShift", "TimeToAnswerQuestion", "TimeToAnswerQuiz", 
    "QuestionDifficulty", "QuestionMedianTime", "QuizMedianTime",
    "Math1", "Math1StdIQ", "Math1Difficulty",
    "Math2", "Math2StdIQ", "Math2Difficulty",
    "Math3", "Math3StdIQ", "Math3Difficulty",
    "Math4", "Math4StdIQ", "Math4Difficulty",
    "Math5", "Math5StdIQ", "Math5Difficulty",
    "Math6", "Math6StdIQ", "Math6Difficulty",
    "Math7", "Math7StdIQ", "Math7Difficulty",
    "Math8", "Math8StdIQ", "Math8Difficulty",
    "YearAnswered", "MonthAnswered", "DayAnswered",
    "HourAnswered", "MinuteAnswered", "QuestionLevel",
    "QuizLength", "#QuestionForStudent"
]]

In [ ]:
# Splitting data into two sets, task1 and task2
# Then exporting train and test data as .csv files
task1_allData = allData.drop(["CorrectAnswer", "AnswerValue"], axis=1)
task2_allData = allData.drop(["IsCorrect"], axis=1)

task1_allData.to_csv('task1_allData.csv', header=True, index=False)
task2_allData.to_csv('task2_allData.csv', header=True, index=False)

public_test_task1_data.to_csv('task1_public_test_data.csv', header=True, index=False)
public_test_task2_data.to_csv('task2_public_test_data.csv', header=True, index=False)

private_test_task1_data.to_csv('task1_private_test_data.csv', header=True, index=False)
private_test_task2_data.to_csv('task2_private_test_data.csv', header=True, index=False)

__________

This section to replace the following columns :
- Math1 through Math8 with MathType
- Math1Difficulty through Math8Difficulty with MathDifficulty
- Math1StdIQ through Math8StdIQ with MathStdIQ

________

In [3]:
task1_allData = pd.read_csv("task1_allData.csv")
task2_allData = pd.read_csv("task2_allData.csv")
task1_public = pd.read_csv("task1_public_test_data.csv")
task1_private = pd.read_csv("task1_private_test_data.csv")
task2_public = pd.read_csv("task2_public_test_data.csv")
task2_private = pd.read_csv("task2_private_test_data.csv")

In [4]:
(task1_allData["MathType"], task1_public["MathType"], task1_private["MathType"], 
 task2_allData["MathType"], task2_public["MathType"], task2_private["MathType"]) = (0, 0, 0, 0, 0, 0)

def mathType(row) :
    for i in range(1, 9) :
        math = "{}{}".format("Math", i)
        if row[math] == 1:
            return i

task1_allData["MathType"] = task1_allData.apply(lambda row: mathType(row), axis = 1)
task1_public["MathType"] = task1_public.apply(lambda row: mathType(row), axis = 1)
task1_private["MathType"] = task1_private.apply(lambda row: mathType(row), axis = 1)

task2_allData["MathType"] = task2_allData.apply(lambda row: mathType(row), axis = 1)
task2_public["MathType"] = task2_public.apply(lambda row: mathType(row), axis = 1)
task2_private["MathType"] = task2_private.apply(lambda row: mathType(row), axis = 1)

In [5]:
(task1_allData["MathDifficulty"], task1_public["MathDifficulty"], task1_private["MathDifficulty"],
 task2_allData["MathDifficulty"], task2_public["MathDifficulty"], task2_private["MathDifficulty"]) = (0, 0, 0, 0, 0, 0)
mathDifficulty = {}

def mathDiff(row) :
    for i in range(1, 9) :
        math = "{}{}".format("Math", i)
        if(row[math] == 1) :
            return mathDifficulty[i]

for i in range(1, 9) :
    math = "{}{}".format("Math", i)
    diff = "{}{}".format(math, "Difficulty")
    mathDifficulty[i] = task1_allData[diff].max()

task1_allData["MathDifficulty"] = task1_allData.apply(lambda row: mathDiff(row), axis = 1)
task1_public["MathDifficulty"] = task1_public.apply(lambda row: mathDiff(row), axis = 1)
task1_private["MathDifficulty"] = task1_private.apply(lambda row: mathDiff(row), axis = 1)

task2_allData["MathDifficulty"] = task2_allData.apply(lambda row: mathDiff(row), axis = 1)
task2_public["MathDifficulty"] = task2_public.apply(lambda row: mathDiff(row), axis = 1)
task2_private["MathDifficulty"] = task2_private.apply(lambda row: mathDiff(row), axis = 1)

In [6]:
(task1_allData["MathStdIQ"], task1_public["MathStdIQ"], task1_private["MathStdIQ"],
 task2_allData["MathStdIQ"], task2_public["MathStdIQ"], task2_private["MathStdIQ"]) = (0, 0, 0, 0, 0, 0)
mathStdIQ = {}
    
def calculateStudentIq(row, iq) :
    if(not(row["UserId"] in mathStdIQ)) :
        mathStdIQ[row["UserId"]] = row[iq]

def getStudentIq(row) :
    if(row in mathStdIQ) :
        return mathStdIQ[row]

for i in range(1, 9) :
    math = "{}{}".format("Math", i)
    iq = "{}{}".format(math, "StdIQ")
    df = task1_allData[task1_allData[math] == 1]
    df.apply(lambda row: calculateStudentIq(row, iq), axis = 1)
    task1_allData["MathStdIQ"] = task1_allData["UserId"].apply(lambda row: getStudentIq(row))
    task1_public["MathStdIQ"] = task1_public["UserId"].apply(lambda row: getStudentIq(row))
    task1_private["MathStdIQ"] = task1_private["UserId"].apply(lambda row: getStudentIq(row))
    task2_allData["MathStdIQ"] = task2_allData["UserId"].apply(lambda row: getStudentIq(row))
    task2_public["MathStdIQ"] = task2_public["UserId"].apply(lambda row: getStudentIq(row))
    task2_private["MathStdIQ"] = task2_private["UserId"].apply(lambda row: getStudentIq(row))

In [7]:
for i in range(1, 9) :
    math = "{}{}".format("Math", i)
    diff = "{}{}".format(math, "Difficulty")
    stdIQ = "{}{}".format(math, "StdIQ")
    task1_allData.drop([math, diff, stdIQ],  inplace = True, axis = 1)
    task1_public.drop([math, diff, stdIQ],  inplace = True, axis = 1)
    task1_private.drop([math, diff, stdIQ],  inplace = True, axis = 1)
    task2_allData.drop([math, diff, stdIQ],  inplace = True, axis = 1)
    task2_public.drop([math, diff, stdIQ],  inplace = True, axis = 1)
    task2_private.drop([math, diff, stdIQ],  inplace = True, axis = 1)

In [9]:
task1_allData.to_csv("task1_allData.csv", header=True, index=False)
task1_public.to_csv("task1_public_test_data_.csv", header=True, index=False)
task1_private.to_csv("task1_private_test_data.csv", header=True, index=False)

task2_allData.to_csv("task2_allData.csv", header=True, index=False)
task2_public.to_csv("task2_public_test_data.csv", header=True, index=False)
task2_private.to_csv("task2_private_test_data.csv", header=True, index=False)

_______

In [ ]:
questionIdEmbedding = pd.read_csv("")
userIdEmbedding = pd.read_csv("")